# Compiling the Data

This notebook is NEW CONTINUING: I am piping in data objects created from other notebooks and combining them.

## Table of Contents
- [1st Section](#Loading-in-the-data) is where I import the files
- [2nd section](#Data-overview) is where I briefly review the data
- [3rd section](#Compiling-the-data) is where I compile the data frames together
- [4th section](#Pickling-the-data-and-creating-a-csv) is where the data are saved and exported
- [Conclusion](#Conclusion) summarizes the notebook

## Loading in the data

In [1]:
# loading packages
import numpy as np
import pandas as pd
import nltk
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [2]:
# load in the data frames

f1 = open('characters_update_df.pkl', 'rb')
characters_df = pickle.load(f1)
f1.close()

f2 = open('conversations_df.pkl', 'rb')
conversations_df = pickle.load(f2)
f2.close()

f3 = open('movies_df.pkl', 'rb')
movies_df = pickle.load(f3)
f3.close

# load in utterances from csv
utterances_df = pd.read_csv('./private/utterances_df.csv', index_col=0)

Let's make sure they all look okay.

In [3]:
characters_df.head()

,character_ID,character_name,movie_ID,movie_title,gender
0,u0,BIANCA,m0,10 things i hate about you,F
2,u2,CAMERON,m0,10 things i hate about you,M
4,u4,JOEY,m0,10 things i hate about you,M
5,u5,KAT,m0,10 things i hate about you,F
6,u6,MANDELLA,m0,10 things i hate about you,F


Link to jnb

In [4]:
conversations_df.head()

,conversation_ID,character1_ID,character2_ID,movie_ID,dialogue
0,0,u0,u2,m0,L194
0,0,u0,u2,m0,L195
0,0,u0,u2,m0,L196
0,0,u0,u2,m0,L197
1,1,u0,u2,m0,L198


Link to jnb

In [5]:
movies_df.head()

,movie_title,movie_year,genres,movie_decade
0,10 things i hate about you,1999,"['comedy', 'romance']",1990
1,1492: conquest of paradise,1992,"['adventure', 'biography', 'drama', 'history']",1990
2,15 minutes,2001,"['action', 'crime', 'drama', 'thriller']",2000
3,2001: a space odyssey,1968,"['adventure', 'mystery', 'sci-fi']",1960
4,48 hrs.,1982,"['action', 'comedy', 'crime', 'drama', 'thrill...",1980


Link to jnb

In [6]:
utterances_df.head()

,line_ID,character_ID,movie_ID,character_name,utterance,sents,tokens,pos_tag,sent_count,token_count,avg_sent_length
0,L1045,u0,m0,BIANCA,They do not!,['They do not!'],"['They', 'do', 'not', '!']","[(They, 'PRON'), (do, 'VERB'), (not, 'PART'), ...",1,4,4.0
1,L1044,u2,m0,CAMERON,They do to!,['They do to!'],"['They', 'do', 'to', '!']","[(They, 'PRON'), (do, 'VERB'), (to, 'PART'), (...",1,4,4.0
2,L985,u0,m0,BIANCA,I hope so.,['I hope so.'],"['I', 'hope', 'so', '.']","[(I, 'PRON'), (hope, 'VERB'), (so, 'ADV'), (.,...",1,4,4.0
3,L984,u2,m0,CAMERON,She okay?,['She okay?'],"['She', 'okay', '?']","[(She, 'PRON'), (okay, 'ADJ'), (?, 'PUNCT')]",1,3,3.0
4,L925,u0,m0,BIANCA,Let's go.,"[""Let's go.""]","['Let', ""'s"", 'go', '.']","[(Let, 'VERB'), ('s, 'PRON'), (go, 'VERB'), (....",1,4,4.0


Link to jnb

They all look good I will review some basic information about the data and then compile the data.

## Data overview

Let's review some basic information about the corpus before diving in.

Talk about how the data relate togheter

In [ ]:
# how many movies in the corpus?
movies_df.movie_title.count()

In [ ]:
# what's the earliest year represented?
movies_df.movie_year.min()

In [ ]:
# what's the latest year represented?
movies_df.movie_year.max()

In [ ]:
# how many characters are in the corpus?
characters_df.character_name.count()

In [ ]:
# how many utterances?
utterances_df.line_ID.count()

In [ ]:
# how many tokens?
utterances_df.token_count.sum()

## Compiling the data

### Discourse

In [8]:
# because I am looking at discourse, I will use the utterances data frame and load other data into it

# adding gender
discourse_df = pd.merge(utterances_df, characters_df)

# adding movie year and decade
discourse_df = pd.merge(discourse_df, movies_df)

# adding conversation data
discourse_df = pd.merge(discourse_df, conversations_df, left_on='line_ID', right_on='dialogue').drop(columns=['character1_ID', 'character2_ID', 'movie_ID_y', 'dialogue'], axis=1)

#rename column
discourse_df.rename(columns={"movie_ID_x": "movie_ID"}, inplace=True)

In [10]:
# there are 267 utterances that are empty, I will remove them
discourse_df = discourse_df[discourse_df.token_count != 0]

In [11]:
discourse_df.head()

,line_ID,character_ID,movie_ID,character_name,utterance,sents,tokens,pos_tag,sent_count,token_count,avg_sent_length,movie_title,gender,movie_year,genres,movie_decade,conversation_ID
0,L1045,u0,m0,BIANCA,They do not!,['They do not!'],"['They', 'do', 'not', '!']","[(They, 'PRON'), (do, 'VERB'), (not, 'PART'), ...",1,4,4.000000,10 things i hate about you,F,1999,"['comedy', 'romance']",1990,24
1,L985,u0,m0,BIANCA,I hope so.,['I hope so.'],"['I', 'hope', 'so', '.']","[(I, 'PRON'), (hope, 'VERB'), (so, 'ADV'), (.,...",1,4,4.000000,10 things i hate about you,F,1999,"['comedy', 'romance']",1990,23
2,L925,u0,m0,BIANCA,Let's go.,"[""Let's go.""]","['Let', ""'s"", 'go', '.']","[(Let, 'VERB'), ('s, 'PRON'), (go, 'VERB'), (....",1,4,4.000000,10 things i hate about you,F,1999,"['comedy', 'romance']",1990,22
3,L872,u0,m0,BIANCA,Okay -- you're gonna need to learn how to lie.,"[""Okay -- you're gonna need to learn how to li...","['Okay', '--', 'you', ""'re"", 'gon', 'na', 'nee...","[(Okay, 'INTJ'), (--, 'PUNCT'), (you, 'PRON'),...",1,13,13.000000,10 things i hate about you,F,1999,"['comedy', 'romance']",1990,21
4,L870,u0,m0,BIANCA,I'm kidding. You know how sometimes you just ...,"[""I'm kidding."", 'You know how sometimes you j...","['I', ""'m"", 'kidding', '.', 'You', 'know', 'ho...","[(I, 'PRON'), ('m, 'AUX'), (kidding, 'VERB'), ...",3,25,8.333333,10 things i hate about you,F,1999,"['comedy', 'romance']",1990,21


In [13]:
discourse_df.describe()

,sent_count,token_count,avg_sent_length,movie_year,movie_decade,conversation_ID
count,304403.000000,304403.000000,304403.000000,304403.000000,304403.000000,304403.000000
mean,1.695433,13.736120,7.856031,1988.106428,1983.434822,41482.442111
std,1.252371,14.712649,5.155597,17.141863,17.204418,23872.389444
min,1.000000,1.000000,1.000000,1927.000000,1920.000000,0.000000
25%,1.000000,5.000000,4.500000,1984.000000,1980.000000,20784.000000
50%,1.000000,9.000000,7.000000,1995.000000,1990.000000,41575.000000
75%,2.000000,17.000000,10.000000,1999.000000,1990.000000,62115.000000
max,45.000000,684.000000,122.000000,2010.000000,2010.000000,83096.000000


Per turn there is a wide range on the token counts, from 1 to 684. There is also a wide range per turn on sentence counts, from 1 to 45. I need to decide if I will include the max values in my analysis to follow. For now they will remain in.

### Tokens

I am looking at token counts to see if there is a difference between turns by gender.

In [ ]:
discourse_df.groupby('gender').token_count.mean()

Only 0.79 difference between average token counts of male and female characters. Characters with unknown gender markers have the longest turns at 14.7. Female characters have the shortest tokens per turn. Female and ambiguous characters fall below the mean and male and unknown are above the mean.

In [ ]:
# raw counts of tokens across the corpus
discourse_df.groupby('gender').token_count.sum()

Raw count doesn't help for comparison across categories, but by looking at this we can see that there are more male characters because the average token count above does not differ as much as the raw counts.

In [ ]:
# what is the shortest turn by token count?
discourse_df.groupby('gender').token_count.min()

Characters across all gender markers have utterances that are only one token long. I expect these will be interjections of some kind.

In [ ]:
discourse_df[discourse_df.token_count==1]

At quick glance, some answers and greetings.

In [ ]:
# what's the longest turn by token count?
discourse_df.groupby('gender').token_count.max()

In [ ]:
# this is the longest token count per utterance
discourse_df[discourse_df.token_count==684]
# seems like it may be a narration intro...it may be removed from analysis

At the total corpus level there are some small differences between gender at the token level. I will factor in movie decade to see if any differences can be detected across time.

### Token and Movie Decade

In [ ]:
discourse_df.groupby(['movie_decade','gender']).token_count.mean()

In [ ]:
discourse_df.groupby(['movie_decade','gender']).token_count.min()
# looks like not all movies have as short of utterances

In [ ]:
discourse_df.groupby(['movie_decade','gender']).token_count.max()

### Sentences

In [ ]:
# how many sentences per turn?
discourse_df.groupby('gender').sent_count.mean()

The average utterance is less than two sentences long. There is not much difference across the categories. On average, male utterances have slightly more sentences. Again, unknown gender has the most (but barely) average sentences per utterance.

In [ ]:
discourse_df.groupby('gender').sent_count.sum()
# more sentences for male characters

This is another view of what we saw above: there are more male characters, which is why the raw sentence count is higher but not the average sentence count.

In [ ]:
# how many tokens per sentence?
discourse_df.groupby('gender').avg_sent_length.mean()
# average sentence length across all genders does not seem to be too wide of a spread

For each gender, sentences have on average around 8 words, although female characters have the shortest averages sentences at 7.67.

In [ ]:
discourse_df.groupby('gender').sent_count.max()
# the longest turn by number of sentences

Commentary

In [ ]:
discourse_df[(discourse_df.sent_count==18) & (discourse_df.gender=='A')]

In [ ]:
discourse_df[(discourse_df.sent_count==33) & (discourse_df.gender=='F')]

In [ ]:
discourse_df[discourse_df.sent_count==45]

In [ ]:
discourse_df[(discourse_df.sent_count==24) & (discourse_df.gender=='unknown')]

The longest utterance by sentence and longest utterance by token count are from male characters.

### Sentences and Movie Decade

How does the sentence level information change by decade?

In [ ]:
discourse_df.groupby(['movie_decade','gender']).sent_count.mean()

COMMENTARY

In [ ]:
discourse_df.groupby(['movie_decade','gender']).sent_count.min()

COMMENTARY

In [ ]:
discourse_df.groupby(['movie_decade','gender']).sent_count.max()

COMMENTARY

In [ ]:
discourse_df.groupby(['movie_decade', 'gender']).avg_sent_length.mean()

COMMENTARY

### Gender/Movie Analysis

In [ ]:
# it may be interesting to see character/gender information by year/decade

gender_df = pd.merge(characters_df, movies_df)

In [ ]:
gender_df.head()

In [ ]:
gender_df.groupby(['movie_decade', 'gender']).count()

In [ ]:
gender_df[gender_df.movie_decade==1920]

Aside from the 1920s, which only had two movies, all other decades have more male characters than female characters.

## Pickling the data and creating a csv

## Conclusion

something